In [1]:
import pandas as pd
import os
from itertools import product
import numpy as np

In [2]:
orders = ['LO', 'NLO', 'N2LO', 'N3LO']
Lambdas = [450, 500]
bodies = ['NN-only', 'NN+3N']
xs = ['0.', '0.5']
fits = {
    (450, 'LO'): [1],
    (500, 'LO'): [6],
    (450, 'NLO'): [2],
    (500, 'NLO'): [7],
    (450, 'N2LO'): [1, 2, 3],
    (500, 'N2LO'): [4, 5, 6],
    (450, 'N3LO'): [7, 8, 9],
    (500, 'N3LO'): [10, 11, 12],
}
data_dir = os.path.expanduser('~/Dropbox/Research/Matter/Matter_Convergence_Analysis/data/')


i = 0
dfs = []

for order, Lambda, body, x in product(orders, Lambdas, bodies, xs):
    if body == 'NN-only':
        filename = os.path.join(data_dir, f'{body}/EOS_x_{x}_{order}_EM{Lambda}new.txt')
        i += 1
        df = pd.read_csv(filename, sep='\t', header=0)
        df.rename(columns={'# kf': 'kf', 'HF_NN': 'MBPT_HF', 'Scnd_NN': 'MBPT_2',
                           'Trd_NN_tot': 'MBPT_3', 'Fth_tot': 'MBPT_4'}, inplace=True)
        df['Lambda'] = Lambda
        df['OrderEFT'] = order
        df['Body'] = body
        df['x'] = float(x)
        df['fit'] = np.nan
        dfs.append(df)
    else:
        for fit in fits[Lambda, order]:
            filename = os.path.join(data_dir, f'{body}/EOS_x_{x}_Ham_{fit}_{order}_EM{Lambda}new.txt')
            i += 1
            df = pd.read_csv(filename, sep='\t', header=0)
            df.rename(columns={'# kf': 'kf', 'HF_tot': 'MBPT_HF', 'Scnd_tot': 'MBPT_2',
                               'Trd_tot': 'MBPT_3', 'Fth_tot': 'MBPT_4'}, inplace=True)
            df['Lambda'] = Lambda
            df['OrderEFT'] = order
            df['Body'] = body
            df['x'] = float(x)
            if order in ['LO', 'NLO']:
                fit = np.nan
            df['fit'] = fit
            dfs.append(df)
            
            # Copy data for easier parsing later on. Given LO and NLO the same fit #s at N2LO and N3LO.
            # if order in ['LO', 'NLO']:
            #     for other_fit in [*fits[Lambda, 'N2LO'], *fits[Lambda, 'N3LO']]:
            #         if other_fit != fit:
            #             df = df.copy() # just in case it would mess with stuff already in dfs
            #             df['fit'] = other_fit
            #             dfs.append(df)
            
    

print('Num files read:', i)
df = pd.concat(dfs, ignore_index=True)

# filename1 = '/Users/jordanmelendez/Dropbox/Research/Matter/Matter_Convergence_Analysis/data/NN-only/EOS_x_0._LO_EM500new.txt'
# filename2 = '/Users/jordanmelendez/Dropbox/Research/Matter/Matter_Convergence_Analysis/data/NN+3N/EOS_x_0._Ham_4_N2LO_EM500new.txt'
# df1 = pd.read_csv(filename1, sep='\t', header=0)
# df1.rename(columns={'# kf': 'kf', 'HF_NN': 'MBPT_HF', 'Scnd_NN': 'MBPT_2',
#                     'Trd_NN_tot': 'MBPT_3', 'Fth_tot': 'MBPT_4'}, inplace=True)
# df2 = pd.read_csv(filename2, sep='\t', header=0)
# df2.rename(columns={'# kf': 'kf', 'HF_tot': 'MBPT_HF', 'Scnd_tot': 'MBPT_2',
#                     'Trd_tot': 'MBPT_3', 'Fth_tot': 'MBPT_4'}, inplace=True)

Num files read: 48


In [3]:
df.dtypes

kf          float64
n           float64
Kin         float64
MBPT_HF     float64
MBPT_2      float64
MBPT_3      float64
MBPT_4      float64
total       float64
Lambda        int64
OrderEFT     object
Body         object
x           float64
fit         float64
dtype: object

In [4]:
df[(df['fit'] == 6) & (df['x'] == 0.)]

,kf,n,Kin,MBPT_HF,MBPT_2,MBPT_3,MBPT_4,total,Lambda,OrderEFT,Body,x,fit
476,1.13972,0.05,16.16071,-7.79485,-1.39880,0.25913,-0.11645,7.10974,500,N2LO,NN+3N,0.0,6.0
477,1.21113,0.06,18.24937,-8.90520,-1.60934,0.30599,-0.12881,7.91201,500,N2LO,NN+3N,0.0,6.0
478,1.27499,0.07,20.22456,-9.89466,-1.83606,0.34515,-0.13557,8.70342,500,N2LO,NN+3N,0.0,6.0
479,1.33302,0.08,22.10753,-10.76819,-2.07685,0.37792,-0.14340,9.49701,500,N2LO,NN+3N,0.0,6.0
480,1.38640,0.09,23.91344,-11.53016,-2.32864,0.40495,-0.14614,10.31345,500,N2LO,NN+3N,0.0,6.0
481,1.43595,0.10,25.65352,-12.18454,-2.58856,0.42865,-0.14671,11.16236,500,N2LO,NN+3N,0.0,6.0
482,1.48231,0.11,27.33645,-12.73510,-2.85174,0.44767,-0.14845,12.04883,500,N2LO,NN+3N,0.0,6.0
483,1.52593,0.12,28.96906,-13.18542,-3.11646,0.46428,-0.14447,12.98699,500,N2LO,NN+3N,0.0,6.0
484,1.56719,0.13,30.55689,-13.53898,-3.37905,0.47854,-0.14520,13.97220,500,N2LO,NN+3N,0.0,6.0
485,1.60639,0.14,32.10448,-13.79920,-3.63704,0.49392,-0.14461,15.01755,500,N2LO,NN+3N,0.0,6.0


In [5]:
df[df['OrderEFT'] == 'LO']

,kf,n,Kin,MBPT_HF,MBPT_2,MBPT_3,MBPT_4,total,Lambda,OrderEFT,Body,x,fit
0,1.13972,0.05,16.16071,-5.56660,-1.93533,0.01619,-0.03168,8.64329,450,LO,NN-only,0.0,NaN
1,1.21113,0.06,18.24937,-6.53108,-2.06096,0.10349,-0.04271,9.71811,450,LO,NN-only,0.0,NaN
2,1.27499,0.07,20.22456,-7.47207,-2.14500,0.17957,-0.05451,10.73255,450,LO,NN-only,0.0,NaN
3,1.33302,0.08,22.10753,-8.39215,-2.19485,0.24304,-0.07415,11.68942,450,LO,NN-only,0.0,NaN
4,1.38640,0.09,23.91344,-9.29313,-2.21891,0.29680,-0.09426,12.60394,450,LO,NN-only,0.0,NaN
5,1.43595,0.10,25.65352,-10.17635,-2.22153,0.33848,-0.11648,13.47764,450,LO,NN-only,0.0,NaN
6,1.48231,0.11,27.33645,-11.04286,-2.20757,0.36954,-0.13399,14.32157,450,LO,NN-only,0.0,NaN
7,1.52593,0.12,28.96906,-11.89349,-2.18011,0.39381,-0.14640,15.14287,450,LO,NN-only,0.0,NaN
8,1.56719,0.13,30.55689,-12.72891,-2.14252,0.41197,-0.16111,15.93632,450,LO,NN-only,0.0,NaN
9,1.60639,0.14,32.10448,-13.54968,-2.09665,0.42016,-0.16840,16.70991,450,LO,NN-only,0.0,NaN


In [6]:
df.to_csv('all_matter_data.csv', index=False)